<a href="https://colab.research.google.com/github/oberljn/order-flow/blob/master/active_per_day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
# Grab the example orders data from a published Google Sheet tab (File > Publish to the web ... > Comma-separated values (.csv))
orders = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vSJqWQejvfq9iLypDVsUc6GLC8ZvJ8TCCTU9n0Nqq2ogxpbHEnsdIWOyJkTXY9K-l2gLeAJJyPJNRx0/pub?gid=0&single=true&output=csv",
    parse_dates=["Start", "End"])

# Data overview
orders.info()
orders.tail()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
Order    56 non-null object
Start    56 non-null datetime64[ns]
End      44 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 1.4+ KB


,Order,Start,End
51,Order 07,2019-01-23,NaT
52,Order 02,2019-01-24,NaT
53,Order 03,2019-01-24,NaT
54,Order 04,2019-01-24,NaT
55,Order 01,2019-01-25,NaT


In [0]:
# Build timeseries dataframe

# Build Timeseries of dates
timeseries_array = [orders.Start.min()]

# Add one day to last entry in array
while timeseries_array[-1] < orders.End.max():
  timeseries_array.append( timeseries_array[-1] + pd.Timedelta("1 days") )

del timeseries_array[timeseries_array.index(max(timeseries_array))]
  
# Add orders count column
order_count = []
for i in timeseries_array:
  order_count.append(None)

timeseries = pd.DataFrame(data = {"Date": timeseries_array, "Orders": order_count})

timeseries.head()

,Date,Orders
0,2018-12-05,None
1,2018-12-06,None
2,2018-12-07,None
3,2018-12-08,None
4,2018-12-09,None


In [0]:
# Count open orders per date in timeseries
for index, row in timeseries.iterrows():
   
  count = 0
  
  for ind, r in orders.iterrows():
    
    cond_1 = r.Start <= row.Date  # Order started on or before date
    cond_2 = r.End >= row.Date    # Order ended on or after date
    cond_3 = r.End == pd.NaT      # Order does not have an end date (remains active)
    
    if cond_1 and (cond_2 or cond_3):
      count += 1

  timeseries.at[index, "Orders"] = count
  
timeseries.info()
timeseries.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
Date      52 non-null datetime64[ns]
Orders    52 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 912.0+ bytes


,Date,Orders
0,2018-12-05,1
1,2018-12-06,1
2,2018-12-07,6
3,2018-12-08,6
4,2018-12-09,6


In [0]:
# Add an SMA to timeseries
# Plot
